In [1]:
#!pip install spotipy lyricsgenius pandas

import spotipy
import lyricsgenius
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import time
import os

In [27]:
client_id = '15f917d7f16f4c7bb1fb590fac0ce9bf'
client_secret = '0aa4cf9038e34be9b3f6b377babb8d8f'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [28]:
# 데이터 저장을 위한 리스트 초기화
artist_name = []
track_name = []
track_popularity = []
track_id = []
artist_id = []
artist_genres = []
artist_followers = []
audio_features_list = []
release_date = []

# 데이터 수집하는 함수
def collect_data(year):
    try:
        for i in range(0, 1000, 50):
            track_results = sp.search(q=year, type='track', limit=50, offset=i)
            for t in track_results['tracks']['items']:
                # 트랙 정보
                artist_info = sp.artist(t['artists'][0]['id'])
                artist_name.append(t['artists'][0]['name'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                artist_id.append(t['artists'][0]['id'])
                release_date.append(t['album']['release_date'])
                artist_genres.append(artist_info['genres'])
                artist_followers.append(artist_info['followers']['total'])            
                # 오디오 피처 정보 수집
                audio_features = sp.audio_features(t['id'])[0]
                if audio_features is not None:
                    audio_features_list.append(audio_features)
                else:
                    print(f"No audio features for track ID: {t['id']}")
                    audio_features_list.append({})
            time.sleep(11)

    except Exception as e:
        print(f"An error occurred: {e}")



한번에 1000곡 밖에 못 가져오는 것 같음   
수기로 year 바꿔주면서 year당 1000곡씩 가져오기?

In [29]:
# 완료된 연도: 2024, 2023, 2022, 2021, 2020, 2019,2018
collect_data('year:2017')

# 수집된 데이터 확인
print(f"Collected {len(audio_features_list)} audio features")
print(f"Collected {len(artist_name)} tracks")

Collected 1000 audio features
Collected 1000 tracks


In [30]:
# 오디오 특징 데이터프레임 생성
audio_features_df = pd.DataFrame(audio_features_list)
audio_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.533,0.907,11,-3.793,1,0.0406,0.0292,0.000000,0.3860,0.700,150.990,audio_features,698eQRku24PIYPQPHItKlA,spotify:track:698eQRku24PIYPQPHItKlA,https://api.spotify.com/v1/tracks/698eQRku24PI...,https://api.spotify.com/v1/audio-analysis/698e...,183160,4
1,0.762,0.604,7,-5.296,0,0.1100,0.1140,0.000000,0.0271,0.847,168.007,audio_features,7yNf9YjeO5JXUE3JEBgnYc,spotify:track:7yNf9YjeO5JXUE3JEBgnYc,https://api.spotify.com/v1/tracks/7yNf9YjeO5JX...,https://api.spotify.com/v1/audio-analysis/7yNf...,97960,4
2,0.551,0.402,11,-7.431,1,0.0262,0.6760,0.000000,0.0928,0.382,103.313,audio_features,2rxQMGVafnNaRaXlRMWPde,spotify:track:2rxQMGVafnNaRaXlRMWPde,https://api.spotify.com/v1/tracks/2rxQMGVafnNa...,https://api.spotify.com/v1/audio-analysis/2rxQ...,193200,4
3,0.695,0.633,8,-4.946,1,0.0599,0.0612,0.000195,0.0926,0.557,96.782,audio_features,3GCdLUSnKSMJhs4Tj6CV3s,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://api.spotify.com/v1/tracks/3GCdLUSnKSMJ...,https://api.spotify.com/v1/audio-analysis/3GCd...,232187,4
4,0.721,0.707,6,-6.364,1,0.0595,0.1810,0.000004,0.1430,0.743,125.005,audio_features,2aPTvyE09vUCRwVvj0I8WK,spotify:track:2aPTvyE09vUCRwVvj0I8WK,https://api.spotify.com/v1/tracks/2aPTvyE09vUC...,https://api.spotify.com/v1/audio-analysis/2aPT...,158206,4


In [31]:
# 트랙 정보 데이터프레임
track_df = pd.DataFrame({
    'artist_name': artist_name,
    'track_name': track_name,
    'track_id': track_id,
    'track_popularity': track_popularity,
    'artist_id': artist_id,
    'artist_genres': artist_genres,
    'artist_followers': artist_followers,
    'release_date': release_date,
})
track_df.head()

,artist_name,track_name,track_id,track_popularity,artist_id,artist_genres,artist_followers,release_date
0,Luke Combs,She Got the Best of Me,698eQRku24PIYPQPHItKlA,83,718COspgdWOnwOFpJHRZHS,"[contemporary country, country]",10973622,2018-06-01
1,Dominic Fike,Babydoll,7yNf9YjeO5JXUE3JEBgnYc,85,6USv9qhCn6zfxlBQIYJ9qs,"[alternative pop rock, pov: indie]",1809637,2018-10-16
2,Luke Combs,Beautiful Crazy,2rxQMGVafnNaRaXlRMWPde,82,718COspgdWOnwOFpJHRZHS,"[contemporary country, country]",10973622,2018-06-01
3,Kendrick Lamar,All The Stars (with SZA),3GCdLUSnKSMJhs4Tj6CV3s,87,2YZyLoL8N0Wb9xBt1NhZWg,"[conscious hip hop, hip hop, rap, west coast rap]",31326158,2018-02-09
4,A$AP Rocky,Sundress,2aPTvyE09vUCRwVvj0I8WK,85,13ubrt8QOOCPljQ2FL1Kca,"[east coast hip hop, hip hop, rap]",14619560,2018-11-20


In [32]:
# 두 데이터프레임 병합
new_data_df = pd.concat([track_df, audio_features_df], axis=1)
new_data_df.head()

,artist_name,track_name,track_id,track_popularity,artist_id,artist_genres,artist_followers,release_date,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Luke Combs,She Got the Best of Me,698eQRku24PIYPQPHItKlA,83,718COspgdWOnwOFpJHRZHS,"[contemporary country, country]",10973622,2018-06-01,0.533,0.907,...,0.3860,0.700,150.990,audio_features,698eQRku24PIYPQPHItKlA,spotify:track:698eQRku24PIYPQPHItKlA,https://api.spotify.com/v1/tracks/698eQRku24PI...,https://api.spotify.com/v1/audio-analysis/698e...,183160,4
1,Dominic Fike,Babydoll,7yNf9YjeO5JXUE3JEBgnYc,85,6USv9qhCn6zfxlBQIYJ9qs,"[alternative pop rock, pov: indie]",1809637,2018-10-16,0.762,0.604,...,0.0271,0.847,168.007,audio_features,7yNf9YjeO5JXUE3JEBgnYc,spotify:track:7yNf9YjeO5JXUE3JEBgnYc,https://api.spotify.com/v1/tracks/7yNf9YjeO5JX...,https://api.spotify.com/v1/audio-analysis/7yNf...,97960,4
2,Luke Combs,Beautiful Crazy,2rxQMGVafnNaRaXlRMWPde,82,718COspgdWOnwOFpJHRZHS,"[contemporary country, country]",10973622,2018-06-01,0.551,0.402,...,0.0928,0.382,103.313,audio_features,2rxQMGVafnNaRaXlRMWPde,spotify:track:2rxQMGVafnNaRaXlRMWPde,https://api.spotify.com/v1/tracks/2rxQMGVafnNa...,https://api.spotify.com/v1/audio-analysis/2rxQ...,193200,4
3,Kendrick Lamar,All The Stars (with SZA),3GCdLUSnKSMJhs4Tj6CV3s,87,2YZyLoL8N0Wb9xBt1NhZWg,"[conscious hip hop, hip hop, rap, west coast rap]",31326158,2018-02-09,0.695,0.633,...,0.0926,0.557,96.782,audio_features,3GCdLUSnKSMJhs4Tj6CV3s,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://api.spotify.com/v1/tracks/3GCdLUSnKSMJ...,https://api.spotify.com/v1/audio-analysis/3GCd...,232187,4
4,A$AP Rocky,Sundress,2aPTvyE09vUCRwVvj0I8WK,85,13ubrt8QOOCPljQ2FL1Kca,"[east coast hip hop, hip hop, rap]",14619560,2018-11-20,0.721,0.707,...,0.1430,0.743,125.005,audio_features,2aPTvyE09vUCRwVvj0I8WK,spotify:track:2aPTvyE09vUCRwVvj0I8WK,https://api.spotify.com/v1/tracks/2aPTvyE09vUC...,https://api.spotify.com/v1/audio-analysis/2aPT...,158206,4


In [33]:
# 기존에 이미 저장되어있는 데이터 불러오기
previous_data_df = pd.read_csv('tracks.csv')
# 새로 수집한 데이터와 합치기
new_data_df = pd.concat([previous_data_df, new_data_df], axis=0, ignore_index=True)
# CSV 파일로 저장
new_data_df.to_csv('tracks.csv', index=False)

2. Genius API   
노래정보 데이터 다 가져오고 가사 수집하기!!!!!

In [ ]:
# Genius API 토큰
genius_api_token = 'mlTKsVm5E6Imvc5ka5qGopA_Y-XggywXVkXd5JtqChpQzBiNOdihlFzhbPfFkCde'
# Initialize Genius API client
genius = lyricsgenius.Genius(genius_api_token)

In [ ]:
# 가사 칼럼 추가할 spotify 데이터 가져오기
genius_df = pd.read_csv('tracks.csv', encoding='utf-8-sig')

In [ ]:
# 가사 가져오는 함수
def get_lyrics_from_genius(track_name, artist_name):
    try:
        song = genius.search_song(track_name, artist_name)
        if song:
            return song.lyrics
        else:
            print(f"No lyrics found for {track_name} by {artist_name}")
            return None
    except Exception as e:
        print(f"An error occurred while fetching lyrics for {track_name} by {artist_name}: {e}")
        return None

In [ ]:
# 가사 정보를 추가하여 데이터프레임 업데이트
genius_df['lyrics'] = genius_df.apply(lambda row: get_lyrics_from_genius(row['track_name'], row['artist_name']), axis=1)

# 결측치 확인
missing_lyrics = genius_df[genius_df['lyrics'].isnull()]
if not missing_lyrics.empty:
    print(f"Failed to retrieve lyrics for {len(missing_lyrics)} tracks")

# 'lyrics' 열의 결측치 개수 확인
missing_lyrics_count = df['lyrics'].isnull().sum()
print(f"Number of missing lyrics: {missing_lyrics_count}")

In [ ]:
# 결측치 처리(삭제)
genius_df.dropna(subset=['lyrics'], inplace=True)

# 가사 정보를 포함하여 데이터프레임 다시 저장
genius_df.to_csv('tracks_with_lyrics.csv', index=False, encoding='utf-8-sig')

print("Data with lyrics saved to tracks_with_lyrics.csv")

In [2]:
"""
df_1 = pd.read_csv('tracks_with_lyrics.csv')
df_2 = pd.read_csv('tracks_with_lyrics_more.csv')

# 새로 수집한 데이터와 합치기
df_3 = pd.concat([df_1, df_2], axis=0, ignore_index=True)
# CSV 파일로 저장
df_3.to_csv('tracks_raw.csv', index=False)
"""

In [3]:
"""
df_3.count()
"""

artist_name         10024
track_name          10024
track_id            10024
track_popularity    10024
artist_id           10024
artist_genres       10024
artist_followers    10024
release_date        10024
danceability         9977
energy               9977
key                  9977
loudness             9977
mode                 9977
speechiness          9977
acousticness         9977
instrumentalness     9977
liveness             9977
valence              9977
tempo                9977
type                 9977
id                   9977
uri                  9977
track_href           9977
analysis_url         9977
duration_ms          9977
time_signature       9977
lyrics              10024
dtype: int64